In [2]:

import torch
import os
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset

import librosa
import librosa.display
import numba.decorators
import numpy as np
import matplotlib.pyplot as plt
from numba.decorators import jit as optional_jit


from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


#PATH = 'C://Projects//keras_talk//keras//intern//dataset//'
PATH = '/content/gdrive/My Drive/dataset/'

train_size = 800
test_size = 200

EPOCHS = 3
BATCH_SIZE = 50


def Y_DATA(y_data):
    for idx in range(y_data.shape[0]):
        y = y_data[idx]
        if y < 0:  y_data[idx] = 10
        else:      y_data[idx] = (y//20)
    return y_data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


###### data normalization

In [19]:

import numpy as np


dataset_dict = { 0 : 'S_left',        1 : 'S_left_phase',
                 2 : 'S_right',       3 : 'S_right_phase',
                 4 : 'clean_left',    5 : 'clean_left_phase',
                 6 : 'clean_right',   7 : 'clean_right_phase',
                 8 : 'idx_drone_end', 9 : 'idx_voice_end',
                10 : 'idx_voice_start'}



numpy_dict = dict()
for n in range(4):
    numpy_name    = dataset_dict[n]
    numpy_dict[n] = np.load( PATH + numpy_name + '.npy' )
    


'''    x_data,       y_data '''
'''(1000,4,257,382), (1000,)'''

x_data = []
for idx in range(1000):
    x_element = []

    for n in range(4):
        x_element.append( numpy_dict[n][:,:,idx] )

    x_element = np.asarray( x_element )
    
    #log scale 변환
    x_element[0] = np.log10( np.abs(x_element[0]) + np.finfo(np.float32).eps )
    x_element[2] = np.log10( np.abs(x_element[2]) + np.finfo(np.float32).eps )


    # normalization
    x_mean = x_element[0].mean()
    x_stdv = x_element[0].std()
    x_element[0] = ( x_element[0] - x_mean ) / x_stdv

    x_mean = x_element[2].mean()
    x_stdv = x_element[2].std()
    x_element[2] = ( x_element[2] - x_mean ) / x_stdv


    x_data.append( x_element )


x_data = np.asarray(x_data)
y_data = Y_DATA( np.load(PATH + 'angle.npy') )
print('done..')

done..


In [20]:

#x_data = x_data.reshape()
#y_data = y_data.reshape()



x_data = torch.from_numpy( x_data ).float().to('cuda')
y_data = torch.from_numpy( y_data ).long().to('cuda')

full_dataset = TensorDataset( x_data, y_data )


train_dataset, valid_dataset = torch.utils.data.random_split( full_dataset, [train_size, test_size])
train_dataset = DataLoader( dataset=train_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last=True)
valid_dataset = DataLoader( dataset=valid_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last=True)


#### ResNet

In [21]:
import torch.nn as nn
import torch.nn.functional as F


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)





class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride



    def forward(self, x):
        identity = x

        out = self.conv1(x)    # 3x3 stride = stride
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)  # 3x3 stride = 1
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x) 

        out += identity
        out = self.relu(out)

        return out






class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x) # 1x1 stride = 1
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = stride
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 planes, plaines * self.expansion
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out



class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=11, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        
        super(ResNet, self).__init__()
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        
        

        self.groups = groups
        self.base_width = width_per_group

        # input : 3 x 224 x 224
        self.conv1 = nn.Conv2d(4, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        
        # output = self.conv1( input )
        # output : 64 x 112 x 112
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        # input : 64 x 112 x 112
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # output : 64 x 56 x 56

        #[layers]
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)


    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)




    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)



def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model



def resnet50(pretrained=False, progress=True, **kwargs):

    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)

##### model train

In [22]:
torch.manual_seed(1)


model = resnet50().to('cuda')

criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

train_loss = []
train_acc  = []


model.train()
for epoch in range(10):
    print('epoch' + str(epoch+1))
    
    for i, (data, label) in enumerate(train_dataset):
        (data, label) = (data.to('cuda'), label.to('cuda'))

        optimizer.zero_grad()
        output = model(data)
  
        loss = F.nll_loss(output, label.reshape(BATCH_SIZE))
        loss.backward()
        optimizer.step()
        
        preds = output.data.max(1)[1]
        corr  = (preds==label.reshape(BATCH_SIZE)).sum().item()
        acc   = corr/BATCH_SIZE*100
        
        train_loss.append(loss.item())
        train_acc.append( acc )
        
        print('\tLoss: {:.3f}\tAcc: {:.3f}'.format(loss.item(), acc))
        

epoch1
	Loss: -0.188	Acc: 12.000
	Loss: -0.223	Acc: 10.000
	Loss: -0.292	Acc: 12.000
	Loss: -0.317	Acc: 6.000
	Loss: -0.263	Acc: 8.000
	Loss: -0.217	Acc: 6.000
	Loss: -0.273	Acc: 6.000
	Loss: -0.377	Acc: 6.000
	Loss: -0.262	Acc: 2.000
	Loss: -0.359	Acc: 14.000
	Loss: -0.388	Acc: 4.000
	Loss: -0.413	Acc: 8.000
	Loss: -0.442	Acc: 6.000
	Loss: -0.408	Acc: 10.000
	Loss: -0.616	Acc: 10.000
	Loss: -0.503	Acc: 8.000
epoch2
	Loss: -0.695	Acc: 16.000
	Loss: -0.550	Acc: 10.000
	Loss: -0.545	Acc: 6.000
	Loss: -0.560	Acc: 8.000
	Loss: -0.631	Acc: 14.000
	Loss: -0.749	Acc: 12.000
	Loss: -0.715	Acc: 12.000
	Loss: -0.753	Acc: 4.000
	Loss: -0.736	Acc: 4.000
	Loss: -0.851	Acc: 12.000
	Loss: -0.777	Acc: 10.000
	Loss: -0.691	Acc: 6.000
	Loss: -0.707	Acc: 2.000
	Loss: -0.975	Acc: 10.000
	Loss: -1.004	Acc: 14.000
	Loss: -0.978	Acc: 8.000
epoch3
	Loss: -0.865	Acc: 8.000
	Loss: -0.811	Acc: 2.000
	Loss: -1.036	Acc: 12.000
	Loss: -0.959	Acc: 4.000
	Loss: -1.069	Acc: 8.000
	Loss: -1.046	Acc: 10.000
	Loss: -1.17